# ML Coding Challenge
## Random Forest Rangers
## Urban, Atilla, Jano

## Setup

In [1]:
import os


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Load Training & Test Data

In [2]:
# mount g-drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# file locations
DATA_PATH = "drive/MyDrive/AIML24"
TRAINING_PATH = DATA_PATH + "/remote_sensing/otherDatasets/sentinel_2/tif"
TEST_PATH = DATA_PATH + "/remote_sensing/otherDatasets/sentinel_2/testset"

In [3]:
# Download Training Data
# !wget https://madm.dfki.de/files/sentinel/EuroSATallBands.zip --no-check-certificate
# !unzip EuroSATallBands.zip
# !mv ds/* "drive/MyDrive/AIML24"

# ☝️Test data must be downloaded manually: put into AIML24/remote_sensing/otherDatasets/sentinel_2/testdata dir

In [4]:
# !unzip "drive/MyDrive/AIML24/remote_sensing/otherDatasets/sentinel_2/testset.zip"
# !mv testset/testset/* "drive/MyDrive/AIML24/remote_sensing/otherDatasets/sentinel_2/testset"

## Define PyTorch Dataset

In [8]:
import glob
len(glob.glob(os.path.join(TEST_PATH,  f"*.npy")))

0

In [9]:
!pip install rasterio

   ---------------------------------------- 0.0/23.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/23.4 MB 1.7 MB/s eta 0:00:14
   - -------------------------------------- 0.6/23.4 MB 6.8 MB/s eta 0:00:04
   - -------------------------------------- 1.0/23.4 MB 9.4 MB/s eta 0:00:03
   - -------------------------------------- 1.0/23.4 MB 9.4 MB/s eta 0:00:03
   - -------------------------------------- 1.0/23.4 MB 9.4 MB/s eta 0:00:03
   - -------------------------------------- 1.0/23.4 MB 9.4 MB/s eta 0:00:03
   - -------------------------------------- 1.0/23.4 MB 9.4 MB/s eta 0:00:03
   -- ------------------------------------- 1.3/23.4 MB 3.5 MB/s eta 0:00:07
   --- ------------------------------------ 2.1/23.4 MB 5.3 MB/s eta 0:00:05
   --- ------------------------------------ 2.1/23.4 MB 5.3 MB/s eta 0:00:05
   ----- ---------------------------------- 3.1/23.4 MB 6.3 MB/s eta 0:00:04
   ----- ---------------------------------- 3.1/23.4 MB 6.3 MB/s eta 0:00:04
   ---

In [35]:
import rasterio as rio
import glob
from torch.utils.data import Dataset
from collections.abc import Callable


def load_img(img_path:str) -> np.ndarray:
  if img_path.split('.')[-1] == "tif":
    with rio.open(img_path, "r") as d:
      img = d.read([1,2,3,4,5,6,7,8,9,10,11,12,13])
      # Assuming bands 2, 3, 4 are the RGB channels (1-based indexing in rasterio)
      # Adjust the indices as necessary based on your data
      # img = d.read([2, 3, 4])
      img = reshape_as_image(img)
  else:
    img = np.load(img_path)
  return img

class SentinelTrain(Dataset):
    def __init__(self, transformations=None):
        self.img_paths = glob.glob(os.path.join(TRAINING_PATH, "*", f"*.tif")) # TODO: remove data limit
        # self.img_paths = glob.glob(os.path.join(TRAINING_PATH, "*", f"*.tif"))
        folder_names = [f.split("/")[-1] for f in glob.glob(TRAINING_PATH + "/*")]
        self.label2ids = {name: id for (id, name) in enumerate(sorted(folder_names))}
        self.ids2label = {v:k for k, v in self.label2ids.items()}
        self.transformations = transformations
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = load_img(img_path)
        
        if self.transformations:
            image = self.transformations(image)
        
        label = img_path.split("/")[-1].split("_")[0]
        one_hot = self.text2oh(label)
        return image, one_hot
    
    def text2oh(self, label):
        one_hot = np.zeros(10)
        one_hot.put(self.label2ids[label], 1)
        return one_hot
    
    def oh2text(self, one_hot):
        idx = np.argmax(one_hot)
        return self.ids2label[idx]
    
def get_id(img_path):
    return img_path.split("/")[-1].split("_")[-1].split(".")[0]

class SentinelTest(Dataset):
    def __init__(self, transformations=None):
        self.img_paths = glob.glob(os.path.join(TEST_PATH,  f"*.npy"))
        self.transformations = transformations
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = load_img(img_path)
        image_id = get_id(img_path)
        
        if self.transformations:
            image = self.transformations(image)
        return image, image_id
    

In [36]:
# Create instances of the Dataset Class for both train & test
import torch
import torchvision.transforms as transforms
from rasterio.plot import reshape_as_image


def bandselect(img):
    return img[:, :, [1,2,3]]

def convert_to_float(img):
    return img.astype(np.float32) / 10000.0


# TODO: improvement -> find global max / min
def l2a_approx(img):
    l2a_bands = img[:,:,[0,1,2,3,4,5,6,7,8,9,11,12]]
    band_min = np.min(l2a_bands, (0,1)) # minimal value per band
    return l2a_bands - band_min # dark object subtraction algo approximation

# Define the transformation functions as lambdas or wrap in Lambda transform
train_transforms = transforms.Compose([
    l2a_approx,
    bandselect,
    convert_to_float
    #   if transform.ToTensor() is active, the error with the type appears
    #   transforms.ToTensor(),
])

train_dataset = SentinelTrain(train_transforms)


test_transforms  = transforms.Compose([
    bandselect,
    convert_to_float

#   if transform.ToTensor() is active, the error with the type appears
#   transforms.ToTensor(),

])
test_dataset = SentinelTest(test_transforms)

print(len(train_dataset))
print(len(test_dataset))

27000
4232


In [37]:
test_dataset.__getitem__(2450)[1]

'3202'

In [ ]:
from torch.utils.data import DataLoader

import numpy as np
import torch


train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)



In [ ]:
# Utility function to display image from dataset
from rasterio.plot import reshape_as_image

def normalize_for_display(band_data):
    """Normalize multi-spectral imagery across bands.
    The input is expected to be in HxWxC format, e.g. 64x64x13.
    To account for outliers (e.g. extremly high values due to
    reflective surfaces), we normalize with the 2- and 98-percentiles
    instead of minimum and maximum of each band.
    """
    band_data = np.array(band_data)
    lower_perc = np.percentile(band_data, 2, axis=(0,1))
    upper_perc = np.percentile(band_data, 98, axis=(0,1))
    print(lower_perc)
    print(upper_perc)
    return (band_data - lower_perc) / (upper_perc - lower_perc)


def print_image(img, label, rgb_bands=[3,2,1])-> None:
  """Displays an image. Indices of bands given by "rgb_bands" will be displayed as RGB in the print
  """
  normalized_img = normalize_for_display(img)
  rgb_img = normalized_img[:, :, rgb_bands]
#   rgb_img = img[:, :, rgb_bands]
  fig, ax = plt.subplots(1, figsize=(5,5))
  ax.imshow((rgb_img * 255).astype(np.uint8), vmin=0, vmax=255)
#   ax.imshow(rgb_img)
  try:
    ax.set_title(f"{train_dataset.ids2label[label]}")
  except:
    ax.set_title(f"unknown")
  ax.axis(False)
  plt.tight_layout()

  plt.show()

In [ ]:
# Example image from train
img, label = train_dataset.__getitem__(3001)
print_image(img, label, [0,1,2])


In [ ]:
# example image from test
img, label = test_dataset.__getitem__(800)
print_image(img, label, [0,1,2])


## Training

In [ ]:
img.shape

train model

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchsummary import torchsummary

class CustomResNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet, self).__init__()
        # Load a pre-trained ResNet model
        self.resnet = models.resnet18(pretrained=True)

        # Modify the first convolutional layer to accept 3 input channels
        # self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet.conv1 = nn.Conv2d(64, 64, kernel_size=(3, 3), padding=(1, 1), bias=False)

        # Replace the final fully connected layer to match the number of classes
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Number of classes in your dataset
num_classes = 10

# Create an instance of the model
model = CustomResNet(num_classes=num_classes)

# If you want to see the model summary for input size of 64x64 RGB images
if torch.cuda.is_available():
    model.cuda()
    torchsummary.summary(model, (64, 64, 3))
else:
    torchsummary.summary(model, (64, 64, 3))

training loop

In [ ]:
import tqdm


from torch.nn.functional import one_hot

# Init collection of training epoch losses
train_epoch_losses = []

# Set the model in training mode
model.train()

num_epochs = 1
mini_batch_size = 2 # size of the mini-batches

cross_entropy = nn.CrossEntropyLoss()

from torch import optim
learning_rate = 0.002
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train for n epochs
for epoch in range(num_epochs):

    # Init collection of mini-batch losses
    train_mini_batch_losses = []

    train_loader_progress = tqdm.tqdm(train_dataloader)


    # Update for each min-batch
    for i, (x,y) in enumerate(train_loader_progress):

        # TODO: Move to transforms
        y = torch.tensor(y)
        y = one_hot(y, 10).double()
        x = torch.tensor(x)


        # Transfer data to compute device
        x, y = x.to(device), y.to(device)

        # Forward pass
        #x = x.float()
        pred = model(x)

        # Reset model's gradients
        model.zero_grad()

        # Compute loss
        loss = cross_entropy(pred, y)

        # Run backward pass
        loss.backward()

        # Update network paramaters
        optimizer.step()

        # Store mini-batch losses
        train_mini_batch_losses.append(loss.data.item())

        train_loader_progress.set_description(f"Loss: {loss.item():0.5f}")

        #np.seterr(divide='ignore', invalid='ignore')

    # Compute epoch loss
    train_epoch_loss = np.mean(train_mini_batch_losses)
    train_epoch_losses.append(train_epoch_loss)

    print(f"Epoch {epoch} - Loss: {train_epoch_loss:0.5f}")

# Save final model
torch.save(model.state_dict(), "checkpoint.pt")

In [ ]:
y.type()

In [ ]:
minibatch = next(enumerate(train_dataloader))

In [ ]:
minibatch[1][0].shape

evaluation loop

In [ ]:
# set model in evaluation mode
model.eval()



In [ ]:
random_imgs = next(iter(test_dataloader))[0].to("cuda")

preds = model(random_imgs)

def onehot2label(onehottensor):
    num = torch.argmax(onehottensor).detach().item()
    return train_dataset.ids2label[num]


In [ ]:
img_path = "drive/MyDrive/AIML24/remote_sensing/otherDatasets/sentinel_2/testset/test_3310.npy"

def get_id(img_path):
    return img_path.split("/")[-1].split("_")[-1].split(".")[0]

get_id(img_path)

In [ ]:
onehot2label(preds[2])

In [ ]:
classArray = []

for i, (imgs, paths) in enumerate(test_dataloader):

    img_ids = [get_id(path) for path in paths]
    preds = model(imgs.to("cuda"))
    preds = list(preds.detach())
    preds = [onehot2label(pred) for pred in preds]
    classArray += list(zip(img_ids, preds))


In [ ]:
len(classArray)

In [ ]:
# Ziel: Array: classArray von (image index, predicted label)

#loop durch test_dataloader (minibatch)
    # pred labels (alle im minibatch auf einmal)
    # find image ids for each pred ()
    # Translate pred to text label
    # append to array

## Submission





>We then create a csv with all our predictions to upload it to kaggle.

In [ ]:
import csv

with open('submission_v1.csv', 'w', newline='') as csvfile:
    # Create a CSV writer object
    writer = csv.writer(csvfile)

    # Write the header row
    writer.writerow(['test_id', 'label'])

    # Write each string to a row with its corresponding index as the test_id
    for i, label in classArray:
        writer.writerow([i, label])